In [1]:
import tensorflow as tf

In [2]:
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


##  Bring SPAM-HAM project

In [3]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sms_data = pd.read_csv('spam.csv', encoding='latin-1')
sms_data.head()

# Remove the unwanted columns

cols_to_delete = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']

for col in cols_to_delete:
    sms_data.drop(col, axis= 1, inplace=True)
    
# take a look at data
sms_data.head()

# Add column names

col_names = ['target','sms']
sms_data.columns = col_names


In [4]:
# Remove punctuation
import string

def removePunc(token):
    return token.translate(str.maketrans('', '', string.punctuation))

sms_data['sms'] = sms_data['sms'].apply(removePunc)


In [5]:
# Remove stop words
from nltk.corpus import stopwords

def removeStop(token):
    sub_list =  [word for word in token.split() if word.lower() not in stopwords.words('english')]
    return ' '.join(sub_list)

sms_data['sms'] = sms_data['sms'].apply(removeStop)


In [6]:
# Apply stemming

from nltk.stem import SnowballStemmer

def stemmer (text):
    text = text.split()
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

In [7]:
sms_data['sms'] = sms_data['sms'].apply(stemmer)


In [8]:
def convertToInt(token):
    if token.lower() == 'spam':
        return 1
    else:
        return 0
    
sms_data['target'] = sms_data['target'].apply(convertToInt)


In [9]:
predictors = sms_data['sms']
target = sms_data['target']

In [10]:
target.value_counts()

0    4825
1     747
Name: target, dtype: int64

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer("english")
tf_idf = vectorizer.fit_transform(predictors)

In [12]:
# Train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tf_idf, target, test_size=0.33, random_state=42)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score

In [14]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier(n_neighbors=49)
mnb = MultinomialNB(alpha=0.2)
dtc = DecisionTreeClassifier(min_samples_split=7, random_state=111)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=31, random_state=111)
abc = AdaBoostClassifier(n_estimators=62, random_state=111)
bc = BaggingClassifier(n_estimators=9, random_state=111)
etc = ExtraTreesClassifier(n_estimators=9, random_state=111)

In [15]:
clfs = {'SVC' : svc,'KN' : knc, 'NB': mnb, 'DT': dtc, 'LR': lrc, 'RF': rfc, 'AdaBoost': abc, 'BgC': bc, 'ETC': etc}

In [16]:
def train_classifier(clf, feature_train, labels_train):    
    clf.fit(feature_train, labels_train)
    
def predict_labels(clf, features):
    return (clf.predict(features))


In [17]:
pred_scores = []
for k,v in clfs.items():
    train_classifier(v, X_train, y_train)
    pred = predict_labels(v,X_test)
    pred_scores.append((k, [accuracy_score(y_test,pred)]))

In [18]:
pred_scores

[('SVC', [0.97444263186514413]),
 ('KN', [0.92822185970636217]),
 ('NB', [0.97824904839586735]),
 ('DT', [0.95758564437194127]),
 ('LR', [0.94344752582925506]),
 ('RF', [0.97389885807504073]),
 ('AdaBoost', [0.96411092985318103]),
 ('BgC', [0.96519847743338771]),
 ('ETC', [0.97335508428493744])]

## Let's apply Deep Learning 

In [19]:
import tensorflow.contrib.learn as skflow

In [22]:
skflow.

SyntaxError: invalid syntax (<ipython-input-22-1c30c8a813a9>, line 1)

In [23]:
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler(with_mean = False ,  with_std = False)


data_scaler.fit(X_train)
X_train = data_scaler.transform(X_train)
X_test = data_scaler.transform(X_test)

In [24]:
X_train

<3733x8037 sparse matrix of type '<class 'numpy.float64'>'
	with 31289 stored elements in Compressed Sparse Row format>

In [25]:
X_train = X_train.todense()

In [26]:
X_test = X_test.todense()

In [27]:
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)

classifier = skflow.DNNClassifier(hidden_units=[50, 100, 200, 100, 50], n_classes=2,feature_columns= feature_columns )
classifier.fit(X_train, y_train, steps=1000, batch_size=32)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000000016C3C6D8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\rnbolla\\AppData\\Local\\Temp\\tmpki6uod9j'}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instruct

C:\Users\rnbolla\AppData\Local\Continuum\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py:254: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b



Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\rnbolla\AppData\Local\Temp\tmpki6uod9j\model.ckpt.
INFO:tensorflow:loss = 0.694043, step = 1
INFO:tensorflow:global_step/sec: 85.3033
INFO:tensorflow:loss = 0.319928, step = 101 (1.169 sec)
INFO:tensorflow:global_step/sec: 97.264
INFO:tensorflow:loss = 0.2483, step =

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._BinaryLogisticHead object at 0x0000000016C3C668>, 'hidden_units': [50, 100, 200, 100, 50], 'feature_columns': (_RealValuedColumn(column_name='', dimension=8037, default_value=None, dtype=tf.float64, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x000000000884AE18>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [28]:
tf_preds = classifier.predict(X_test)

Instructions for updating:
Please switch to predict_classes, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


C:\Users\rnbolla\AppData\Local\Continuum\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py:254: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Restoring parameters from C:\Users\rnbolla\AppData\Local\Temp\tmpki6uod9j\model.ckpt-1000


In [29]:
tf_preds = list(tf_preds)
accuracy_score(y_test,tf_preds)

0.9755301794453507

In [30]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,tf_preds))

             precision    recall  f1-score   support

          0       0.98      0.99      0.99      1587
          1       0.96      0.86      0.91       252

avg / total       0.98      0.98      0.97      1839



In [31]:
pd.DataFrame(tf_preds)[0].value_counts()

0    1614
1     225
Name: 0, dtype: int64

In [32]:
print(confusion_matrix(y_test,tf_preds))

[[1578    9]
 [  36  216]]
